# Baseline tarea 1

-----------------------------




## Importar librerías y utiles

In [28]:
import os
import random
import shutil
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.casual import TweetTokenizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import (confusion_matrix,
                             cohen_kappa_score,
                             classification_report,
                             accuracy_score,
                             roc_auc_score)
from sklearn.model_selection import train_test_split

from utils import auc 




In [29]:
# setear semillas
SEED = 8080
random.seed(SEED)
np.random.seed(SEED)

## Datos

### Obtener los datasets desde el github del curso

In [30]:
base_url = 'https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data'
col_names = ['id', 'tweet', 'class', 'sentiment_intensity']
sentiments = ['anger', 'fear', 'joy', 'sadness']
split_names = ['train', 'target']
train, target = [
    {
        sentiment : pd.read_csv(
            f"{base_url}/{split_name}/{sentiment}-{split_name}.txt",
            sep='\t',
            names=col_names
        )
        for sentiment in sentiments
    }
    for split_name in split_names
]

### Analizar los datos 

Imprimir la cantidad de tweets de cada dataset, según su intensidad de sentimiento

In [31]:
def get_group_dist(group_name, train):
    print(group_name, "\n",
          train[group_name].groupby('sentiment_intensity').count())
    print("-" * 20, '\n')


for key in train:
    get_group_dist(key, train)

anger 
                       id  tweet  class
sentiment_intensity                   
high                 163    163    163
low                  161    161    161
medium               617    617    617
-------------------- 

fear 
                       id  tweet  class
sentiment_intensity                   
high                 270    270    270
low                  288    288    288
medium               699    699    699
-------------------- 

joy 
                       id  tweet  class
sentiment_intensity                   
high                 195    195    195
low                  219    219    219
medium               488    488    488
-------------------- 

sadness 
                       id  tweet  class
sentiment_intensity                   
high                 197    197    197
low                  210    210    210
medium               453    453    453
-------------------- 



## Clasificar

### Dividir el dataset en entrenamiento y prueba

In [32]:
def split_dataset(dataset):
    # Dividir el dataset en train set y test set
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        test_size=0.33,
#         random_state=8080,
    )
    return X_train, X_test, y_train, y_test

### Definir el clasificador

Consejo para el vectorizador: investigar los modulos de `nltk`, en particular, `TweetTokenizer`, `mark_negation`. También, el parámetro ngram_range para clasificadores no bayesianos.

Consejo para el clasificador: investigar otros clasificadores mas efectivos que naive bayes. Ojo q naive bayes no debería usarse con n-gramas, ya que rompe el supuesto de independencia.


In [33]:
# Definimos el pipeline con el vectorizador y el clasificador.
def get_classifier():
     # Inicializamos el Vectorizador para transformar las oraciones a BoW 
    vectorizer = CountVectorizer(tokenizer=TweetTokenizer().tokenize)
    
    # Inicializamos el Clasificador.
    naive_bayes = MultinomialNB()
    
    # Establecer el pipeline.
    text_clf = Pipeline([('vect', vectorizer), ('clf', naive_bayes)])
    return text_clf

### Definir evaluación

Esta función imprime la matriz de confusión, el reporte de clasificación y las metricas usadas en la competencia:


- `auc`
- `kappa`
- `accuracy`

In [34]:
def evaulate(predicted, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador. 
    # (que comunmente, es distinto a ['low', 'medium', 'high'])
    predicted_labels = [labels[np.argmax(item)] for item in predicted]
    
    # Confusion Matrix
    print('Confusion Matrix for {}:\n'.format(key))

    # Classification Report
    print(
        confusion_matrix(y_test,
                         predicted_labels,
                         labels=['low', 'medium', 'high']))

    print('\nClassification Report')
    print(
        classification_report(y_test,
                              predicted_labels,
                              labels=['low', 'medium', 'high']))

    # AUC
    print("auc: ", auc(y_test, predicted))

    # Kappa
    print("kappa:", cohen_kappa_score(y_test, predicted_labels))

    # Accuracy
    print("accuracy:", accuracy_score(y_test, predicted_labels), "\n")

    print('------------------------------------------------------\n\n')

### Ejecutar el clasificador para cierto dataset

Clasifica un dataset. Retorna el modelo ya entrenado mas sus labels asociadas.


In [35]:
def classify(dataset, key):

    X_train, X_test, y_train, y_test = split_dataset(dataset)
    text_clf = get_classifier()

    # Entrenar el clasificador
    text_clf.fit(X_train, y_train)

    # Predecir las probabilidades de intensidad de cada elemento del set de prueba.
    predicted = text_clf.predict_proba(X_test)

    # Obtener las clases aprendidas.
    learned_labels = text_clf.classes_

    # Evaluar
    evaulate(predicted, y_test, learned_labels)
    return text_clf, learned_labels

### Ejecutar el clasificador por cada dataset


In [36]:
classifiers = []
learned_labels_array = []

# Por cada llave en train ('anger', 'fear', 'joy', 'sadness')
for key in train:
    classifier, learned_labels = classify(train[key], key)
    classifiers.append(classifier)
    learned_labels_array.append(learned_labels)

Confusion Matrix for anger:

[[  7  49   1]
 [  4 184  14]
 [  0  42  10]]

Classification Report
              precision    recall  f1-score   support

         low       0.64      0.12      0.21        57
      medium       0.67      0.91      0.77       202
        high       0.40      0.19      0.26        52

    accuracy                           0.65       311
   macro avg       0.57      0.41      0.41       311
weighted avg       0.62      0.65      0.58       311

auc:  0.47482407863488946
kappa: 0.1282743858933849
accuracy: 0.6463022508038585 

------------------------------------------------------


Confusion Matrix for fear:

[[ 19  83   1]
 [ 10 195  17]
 [  0  64  26]]

Classification Report
              precision    recall  f1-score   support

         low       0.66      0.18      0.29       103
      medium       0.57      0.88      0.69       222
        high       0.59      0.29      0.39        90

    accuracy                           0.58       415
   macro avg

## Predecir target set

In [37]:
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset.tweet), columns=labels)
    # Agregar ids
    predicted['id'] = dataset.id.values
    # Reordenar
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted

### Ejecutar la predicción y guardar archivos.

In [38]:
predicted_target = {}

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

for idx, key in enumerate(target):
    # Predecir el target set
    predicted_target[key] = predict_target(target[key], classifiers[idx],
                                           learned_labels_array[idx])
    # Guardar predicciones
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')